# 6.Scoring [Task1]

In [26]:
import pandas as pd
import folium
from haversine import haversine
import numpy as np
import seaborn as sns
import json
import os
import geopandas as gpd
import pathlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor

In [19]:
#Font
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
matplotlib.rcParams['axes.unicode_minus']=False

font_fname = '/opt/app-root/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family
#sns.set(font=font_family)

In [2]:
input_path = pathlib.Path('./data')

## 1. 어린이 교통사고 위험지역 후보군 추출  
- 사고 격자들 중, 기존 어린이 보호구역과 겹치지 않는 격자만 후보로 선정

In [3]:
어린이보호구역 = pd.read_csv(input_path.joinpath('9.오산시_어린이보호구역.csv'),encoding="utf-8")
법정경계_읍면동 = gpd.read_file(input_path.joinpath('33.오산시_법정경계(읍면동).geojson'),encoding="utf-8")
어린이교통사고_격자 = gpd.read_file(input_path.joinpath('2.오산시_어린이교통사고_격자.geojson'),encoding="utf-8")

In [4]:
어린이교통사고_격자['cut']=0
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=2]=1
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=5]=2
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=7]=3
어린이교통사고_격자['cut'][어린이교통사고_격자.accident_cnt>=10]=4

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the 

In [8]:
def check_overlaps(row):
    global 어린이보호구역
    
    if not row.cut:
        return False

    for row2 in 어린이보호구역.iloc:
        [lng,lat] = row2[-2:]
        
        if haversine((lat,lng),row['geometry'].centroid.coords[0][::-1],unit="m") <= 300:
            return True
    return False
            
어린이교통사고_격자['overlapped'] = 어린이교통사고_격자.apply(check_overlaps,axis=1)
후보_격자 = 어린이교통사고_격자[(어린이교통사고_격자.cut > 0)&(어린이교통사고_격자.overlapped == False)]
후보_격자.head(2)

,gid,accident_cnt,geometry,cut,overlapped
1665,다사593100,2,"MULTIPOLYGON (((127.04144 37.18790, 127.04144 ...",1,False
1738,다사594093,2,"MULTIPOLYGON (((127.04261 37.18159, 127.04260 ...",1,False


> __총 32개 격자지역 선정__

In [11]:
print("선정된 후보 격자 수 :",후보_격자.shape[0])

선정된 후보 격자 수 : 32


> __후보 지역 시각화__  

In [13]:
center=[37.164584, 127.050451]
zoom =12.5

m = folium.Map(location=center, zoom_start=zoom)

folium.GeoJson(
    법정경계_읍면동,
    name = "동 경계 [검은 배경]",
    style_function=lambda feature: {
        'color': '#000000',
        'weight': 2,
        'opacity':0.2,
    }
).add_to(m)

for line in 어린이보호구역.iloc:
    [lng,lat] = line[-2:]
    
    folium.Circle(
    location = [lat,lng],
    radius=300,
    stroke=False,
    fill=True,
    fill_color='green',
    fill_opacity=0.6).add_to(m)
    
folium.GeoJson(
    어린이교통사고_격자[어린이교통사고_격자.cut > 0],
    name = '교통사고 2회 이상 [빨간 격자]',
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3,
        'opacity':0.7,
    }
).add_to(m)

folium.GeoJson(
    후보_격자,
    name = '후보 격자 [노란 격자]',
    style_function=lambda feature: {
        'color': 'yellow',
        'weight': 3,
        'opacity':0.7,
    }
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

## 2. 앞서 확인한 Best Model 생성 (AdaBoost) & 학습 및 예측

In [18]:
total = pd.read_csv(input_path.joinpath('total_sub.csv'),index_col='Unnamed: 0')
total.sample(6) #(345,81)

,accident_cnt,고령인구_count_100,생산가능인구_count_100,신호등_sum,유동인구_sum,유소년인구_count_100,유소년인구_count_200,주정차_count_300,차량등록_count_200,학원수_count_100,...,학구.raw오산원일초통학구역,학구.raw오산초가수초공동통학구역,학구.raw오산초통학구역,학구.raw운산초통학구역,학구.raw운암초통학구역,학구.raw원동초통학구역,학구.raw필봉초통학구역,학구.raw화성초통학구역,정류장_count_200,총인구_count_100
156,1,27,143,5,32.152583,19,64,10,0,0,...,0,0,0,0,0,0,0,0,7,189
125,1,26,98,2,62.651583,0,20,7,0,0,...,0,0,0,0,0,0,0,0,4,137
79,2,0,0,2,14.919667,0,7,4,0,0,...,0,0,0,0,0,0,0,0,4,0
158,3,0,0,5,23.146735,0,73,7,0,0,...,0,0,0,0,0,0,0,0,4,0
313,1,0,0,0,0.482433,0,0,3,0,0,...,0,0,0,0,0,0,0,0,3,0
32,1,0,49,5,118.265667,0,26,9,0,1,...,0,0,0,0,0,0,1,0,4,51


### __Generate X , Y data__

In [22]:
X = pd.DataFrame(total.drop(['accident_cnt'],axis=1))
cols = X.columns
Y = pd.Series(total["accident_cnt"])
X.shape , Y.shape

((345, 80), (345,))

### __Train | Test set split__

In [23]:
scaler = StandardScaler()
m_scaler = MinMaxScaler()
X = scaler.fit_transform(X)

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=2021)
x_train.shape , x_test.shape

((276, 80), (69, 80))

### __Make validator__

In [25]:
my_scorer = make_scorer(mean_squared_error,greater_is_better=False)

kf = KFold(n_splits=10,shuffle=True,random_state=2021)

### __Model Train__

In [27]:
ada = AdaBoostRegressor(n_estimators=200,learning_rate=0.1,loss="exponential",random_state=2021)
ada.fit(x_train, y_train)

Rscore = ada.score(x_test,y_test)
score = cross_val_score(ada,x_test,y_test,cv=kf,scoring=my_scorer)  
print("MSE :",score.mean()*(-1))

MSE : 0.47954329369814125


### __Predict Accidents__

In [31]:
pred = ada.predict(X)

# 3. 예측 사고건수로 부터 실제 발생 건수 비교를 통한 위험도 계산  
- 위험도 : 예측 사고건수 - 실제 발생건수

In [30]:
모든_격자 = pd.read_csv(input_path.joinpath("data_for_task1.csv"))
모든_격자.head(2)

,accident_cnt,gid,고령인구_count_100,생산가능인구_count_100,신호등_sum,유동인구_sum,유소년인구_count_100,유소년인구_count_200,주정차_count_300,차량등록_count_200,...,통학구__오산대원초통학구역,통학구__오산원당초통학구역,통학구__오산원일초통학구역,통학구__오산초가수초공동통학구역,통학구__오산초통학구역,통학구__운산초통학구역,통학구__운암초통학구역,통학구__원동초통학구역,통학구__필봉초통학구역,통학구__화성초통학구역
0,1,다사561083,0.0,0.0,0,43.435676,0.0,0.0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,1,다사564096,0.0,0.0,0,9.633417,0.0,0.0,2,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
total['gid'] = 모든_격자.gid
total['pred_score'] = pred

In [38]:
idx=[]

for row in 후보_격자.iloc:
    for i,line in enumerate(total.iloc):
        if row.gid==line.gid:
            idx.append(i)
            
후보_격자_위험도 = total.iloc[idx].copy()
후보_격자_위험도['위험도']=후보_격자_위험도.pred_score-후보_격자_위험도.accident_cnt
후보_격자_위험도.head(2)

,accident_cnt,고령인구_count_100,생산가능인구_count_100,신호등_sum,유동인구_sum,유소년인구_count_100,유소년인구_count_200,주정차_count_300,차량등록_count_200,학원수_count_100,...,학구.raw운산초통학구역,학구.raw운암초통학구역,학구.raw원동초통학구역,학구.raw필봉초통학구역,학구.raw화성초통학구역,정류장_count_200,총인구_count_100,gid,pred_score,위험도
25,1,0,0,0,117.510833,0,104,5,0,0,...,0,0,0,0,0,4,0,다사593100,1.445946,0.445946
28,1,176,1242,0,36.420667,264,550,6,0,0,...,0,0,0,0,0,4,1682,다사594093,1.500000,0.500000


# 4. 위험도 우선순위로 최종 20 격자지역 선정 및 저장

In [39]:
후보_격자_위험도.to_csv(input_path.joinpath('ada_totalscore.csv'),index=False)
최종선정구역 = 후보_격자_위험도.sort_values(by='위험도').head(20)
최종선정구역.to_csv(input_path.joinpath('ada_top_20.csv'),index=False)